# 24_RPeaks_Robustos

Detecção de R-peaks Pan‑Tompkins-like no traçado 1D extraído de uma derivação.

In [ ]:
import numpy as np
from PIL import Image

from cv.grid_detect import estimate_grid_period_px
from cv.rpeaks_from_image import estimate_px_per_sec, extract_trace_centerline, smooth_signal
from cv.rpeaks_robust import pan_tompkins_like
from cv.segmentation import find_content_bbox
from cv.segmentation_ext import segment_layout

img = Image.open('../samples/ecg_images/synthetic_12lead.png').convert('L')
arr = np.asarray(img)
bbox = find_content_bbox(arr)
seg = segment_layout(arr, '3x4', bbox=bbox)
lab2box = {d['lead']: d['bbox'] for d in seg}
x0,y0,x1,y1 = lab2box['II']
crop = arr[y0:y1,x0:x1]
trace = smooth_signal(extract_trace_centerline(crop), win=11)
pxmm = (estimate_grid_period_px(np.asarray(Image.open('../samples/ecg_images/synthetic_12lead.png').convert('RGB'))).get('px_small_x') or 10)
pxsec = estimate_px_per_sec(pxmm, 25.0)
res = pan_tompkins_like(trace, pxsec)
len(res['peaks_idx']), res['params']